In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 0. 환경변수 및 패키지

In [45]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_ollama import ChatOllama
import os
import re
import json
import ollama

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage

from pinecone.exceptions import NotFoundException

from typing import List, Optional, Dict, Any
from __future__ import annotations
from collections import defaultdict

load_dotenv()

True

In [14]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDIGN_MODEL = 'text-embedding-3-large' 
UPSTAGE_EMBEDDIGN_MODEL = 'solar-embedding-1-large-passage'

PINECONE_INDEX_NAME_LAW = 'law-index-final'
PINECONE_INDEX_NAME_RULE = 'rule-index-final'
# PINECONE_INDEX_NAME_CASE = 'case-index-final'

COHERE_API_KEY = os.getenv('COHERE_API_KEY')

# 1. 함수

In [47]:
def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전
    Returns:
        dict: {카테고리명: {키워드: 가중치}}
    """
    return {
        # 1. 보증금 보호 (가장 치명적인 리스크)
        '보증금_대항력': {
            '보증금': 3, '전세금': 2, '보증금반환': 3, '반환청구': 2,
            '대항력': 3, '전입신고': 2, '주민등록': 2, '점유': 2, '인도': 2,
            '우선변제권': 3, '최우선변제': 3, '소액임차인': 3, '배당': 2, '경매': 2, '공매': 2,
            '확정일자': 3, '확정일자부': 2,'임차권등기': 3, '임차권등기명령': 3,
            '반환보증': 2, '보증보험': 2, 'HUG': 1, '보증보험': 2,
            '대항요건': 3,'제삼자': 2,'효력': 1,'효력발생': 2, '입주': 1,
            '거주': 1,'주택인도': 2, '임대차계약증서': 2,'환가대금': 2,
            '후순위권리자': 2,'변제': 1,'배당요구': 2,'임차권등기명령': 3,'임대차등기': 2,
            '지방법원': 2,'등기촉탁': 1,'등기신청': 1,
        },

        # 2. 계약 갱신 및 거절 (가장 빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신요구권': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '재계약': 2, '존속기간': 2, '2년': 1,
            '실거주': 3, '거절사유': 2, '손해배상': 2,
            '연체': 2, '2기': 2, '의무위반': 2,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1, '계약갱신청구권': 3,
            '갱신거절통지': 3,'계약조건변경': 2, '계약만료': 2,'직계존속': 2,
            '직계비속': 2,'직계존비속': 2, '중개보수': 1,
        },

        # 3. 계약 해지 및 종료 (탈출 전략)
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },

        # 4. 차임(월세) 및 보증금 증감
        '임대료_증감': {
            '월세전환': 3, '반전세': 2, '차임': 3, '월세': 2, '증액': 3, 
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2, '감액': 2, '인상': 2, '1년': 2, '1년 이내': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1, '전환되는 금액': 2,
            '초과차임': 2, '반환': 2, '증액청구': 3,'증액비율': 2, '차임액': 2,
            '보증금증액': 2, '차임연체': 2, '2기차임': 2,'연체': 1,
        },

        # 5. 유지보수 및 원상회복 (민법 영역)
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2, '유지': 2, '하자': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },

        # 6. 전세사기 및 권리 리스크 (독소조항 탐지용)
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3, '신탁부동산': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3, 
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2, '등기부등본': 2,
            '말소기준권리': 3, '채권최고액': 2, '가등기': 2, '전세권': 2, '전세권등기': 2,
            '소유권이전등기': 2,'말소': 2, '배당요구': 2,'건축물대장': 2, '보존등기': 2, '이행강제금': 1,
        },

        # 7. 행정 절차 (Rules Index 연결)
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1, '등기부등본': 3,
            '건축물대장': 3,'중개대상물확인설명서': 2,'공인중개사': 1,'중개수수료': 2, '중개보수': 2,
        },

        # 8. 소송 및 분쟁 해결 (최후 수단)
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3, '조정': 3, '조정부': 2,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2, '조정신청': 3, '신청': 2, '각하': 2,
            '조정절차': 3, '송달': 2, '서면': 2, '구두': 2,
            '처리기간': 2, '60일': 2, '30일': 1,
            '심사관': 1, '조사관': 1, '사무국': 1,
            '조정신청': 3,'조정안': 2,'조정서': 2, '조정성립': 2,'인도명령': 2,'명도소송': 2,
            
            # 소송/집행 트리거
            '지급명령': 3, '소송': 3, '판결': 2, '집행권원': 2,
            '경매': 3, '공매': 2, '내용증명': 2, '손해배상': 3, '지연이자': 2,

            # 시행령(원문 표기): 조정 대상 범위 + 신청 절차 상세
            '제22조(조정위원회의 심의ㆍ조정 사항)': 3,
            '제29조(조정부의 구성 및 운영)': 1,
            '제30조(조정의 신청)': 2,
            '제31조(조정신청인에게 안내하여야 할 사항)': 1,
            '제32조(조정서류의 송달 등)': 1,
        },
        
        # 9. 임차권 승계 및 가족 
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        },
        
        # 10. 전대/양도
        '전대_양도': {
            '전대': 3, '전차': 2, '전대차': 2, '임대인의 동의': 3,
            '양도': 2, '임차권 양도': 2
        },

        # 0. 적용 범위/예외(필요 시 사용)
        '적용범위_예외': {
            '적용범위': 3, '주거용': 2, '주택 일부': 2, '주거 외 목적': 2,
            '일시사용': 3, '일시사용하기 위한 임대차': 3,
            '미등기': 3, '미등기 전세': 3, '전세계약': 2
        },
    }


def categorize_content(content, top_k=None):
    '''
    내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters : 
        - content : 분류할 텍스트 내용
        - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
        
    Returns :
        - 카테고리 리스트(점수 높은 순)
    '''
    category_keywords = get_law_category()
    category_scores = {}
    
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0 :
            category_scores[category] = score
    
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x : x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    
    # 매칭되는 카테고리가 없으면 '기타' 반환
    if not all_categories:
        all_categories = ['기타']
        
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    return all_categories


# =====================
# 참조조항을 추출하는 함수
# =====================

_ARTICLE_RE = re.compile(r"(?:제\s*)?(\d+)\s*조(?:\s*의\s*(\d+))?")

def _law_name_from_src_title(src_title: str) -> str:
    if not src_title:
        return "알수없음"
    s = str(src_title).strip()
    if not s:
        return "알수없음"
    # 첫 '(' 이전을 법령명으로 사용
    name = s.split("(", 1)[0].strip()
    return name if name else "알수없음"

def _split_article_field(article_field) -> list[str]:
    if not article_field or article_field == "조항없음":
        return []
    s = str(article_field)
    s = s.replace("「", "").replace("」", "").strip()
    if not s:
        return []
    # ",", ";", 줄바꿈, "·" 등 다양한 구분자 대응
    tokens = re.split(r"[,\n;·]+", s)
    return [t.strip() for t in tokens if t and t.strip()]

def _parse_article_token(token: str):
    if not token:
        return None
    token = re.sub(r"\s+", "", token)
    m = _ARTICLE_RE.search(token)
    if not m:
        return None
    main = int(m.group(1))
    sub = int(m.group(2)) if m.group(2) else 0
    return (main, sub)

def _format_article_key(key):
    main, sub = key
    return f"제{main}조" if sub == 0 else f"제{main}조의{sub}"

def extract_articles_from_docs(documents):
    """
    검색된 문서들(documents)에서 article 정보를 추출하여
    '법령명: 제x조,제y조 | 법령명2: ...' 형태로 반환
    """
    # law_name -> set( (main,sub) )
    buckets = defaultdict(set)
    # law_name -> best priority(작을수록 우선)
    law_priority = {}

    for doc in documents:
        meta = doc.metadata or {}

        law = _law_name_from_src_title(meta.get("src_title") or meta.get("law_name") or meta.get("source"))
        pr = meta.get("priority", None)
        try:
            pr_int = int(pr) if pr is not None else None
        except Exception:
            pr_int = None

        if pr_int is not None:
            prev = law_priority.get(law)
            if prev is None or pr_int < prev:
                law_priority[law] = pr_int

        for tok in _split_article_field(meta.get("article", "조항없음")):
            key = _parse_article_token(tok)
            if key is not None:
                buckets[law].add(key)

    if not buckets:
        return "(참조 조항 없음)"

    # 법령 블록 정렬: priority(있으면) -> 법령명
    def law_sort_key(law_name: str):
        p = law_priority.get(law_name)
        return (p if p is not None else 9999, law_name)

    parts = []
    for law in sorted(buckets.keys(), key=law_sort_key):
        keys = sorted(buckets[law])
        if not keys:
            continue
        parts.append(f"{law} " + ",".join(_format_article_key(k) for k in keys))

    return " | ".join(parts) if parts else "(참조 조항 없음)"



# ====================================
# 검색된 document를 텍스트로 변환하는 함수
# ====================================

def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])


# ===========================
# title+text 기반의 rerank함수
# ===========================

import cohere
import numpy as np

def rerank_by_title(query: str, documents: list, top_k: int = 4):
    '''
    cohere의 rerank API를 사용한 문서 재정렬
    '''
    # cohere client 초기화
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    # 0건이면 rerank 호출하지 말고 그대로 반환
    if not documents:
        return []

    # top_k도 0이면 그대로
    if not top_k or top_k <= 0:
        return documents

    docs_text = [
        (doc.metadata.get("src_title", "") or "") + " " + (doc.page_content or "")
        for doc in documents
    ]

    # documents가 비어있지 않아도, docs_text가 전부 빈문자면 Cohere가 싫어할 수 있어서 방어
    if not any(t.strip() for t in docs_text):
        return documents[:top_k]

    results = co.rerank(
        model="rerank-multilingual-v3.0",
        query=query,
        documents=docs_text,
        top_n=min(top_k, len(docs_text)),
    )

    idxs = [r.index for r in results.results]
    return [documents[i] for i in idxs]


# ==========================================
# 검색된 document를 LLM용 context로 변환 (개선)
# ==========================================

def format_documents_with_meta(documents, max_len: int = 500):
    """
    프롬프트.txt 구조 반영:
    - 법령명 / 조문 / priority 명시
    - LLM이 법적 근거를 명확히 인식하도록 설계
    """
    context_parts = []

    for i, doc in enumerate(documents, 1):
        meta = doc.metadata or {}

        law_name = meta.get("law_name") or meta.get("src_title") or meta.get("source", "알 수 없는 법령")
        article = meta.get("article", "")
        priority = meta.get("priority", "?")

        content = doc.page_content.strip()
        if len(content) > max_len:
            content = content[:max_len].rstrip() + "…"

        context_parts.append(
            f"""[문서 {i}]
- 법령명: {law_name}
- 조문: {article}
- 우선순위(priority): {priority}
- 내용:
{content}
"""
        )

    return "\n\n".join(context_parts)

## Query Planner

In [27]:
def extract_json_object(text: str) -> Optional[Dict[str, Any]]:
    """
    LLM 출력에서 가장 바깥 JSON object({...}) 하나를 찾아 파싱.
    실패하면 None.
    """
    if not text:
        return None

    # 가장 단순하지만 실전에 강한 방식: 첫 { 와 마지막 } 사이를 후보로
    first = text.find("{")
    last = text.rfind("}")
    if first == -1 or last == -1 or last <= first:
        return None

    candidate = text[first:last+1].strip()
    try:
        return json.loads(candidate)
    except Exception:
        # 혹시 코드블록 ```json ... ``` 형태 제거 후 재시도
        candidate2 = re.sub(r"```(?:json)?\s*|\s*```", "", candidate, flags=re.IGNORECASE).strip()
        try:
            return json.loads(candidate2)
        except Exception:
            return None

        
def validate_and_normalize_plan(plan: Dict[str, Any]) -> Dict[str, Any]:
    """
    최소한의 규칙으로 plan을 정리하고, 사용할지 말지 결정하는 플래그 포함해서 반환.
    반환 예:
      {
        "use_plan": True/False,
        "confidence": 0.0~1.0,
        "queries": [...],
        "issues": [...],
        "law_types": [...],
        "need_facts": [...],
        "reason": "..."
      }
    """
    out = {
        "use_plan": False,
        "confidence": 0.0,
        "queries": [],
        "issues": [],
        "law_types": [],
        "need_facts": [],
        "reason": ""
    }
    if not isinstance(plan, dict):
        out["reason"] = "plan_not_dict"
        return out

    # 필드 읽기
    confidence = plan.get("confidence", 0)
    queries = plan.get("queries", [])
    issues = plan.get("issues", [])
    law_types = plan.get("law_types", [])
    need_facts = plan.get("need_facts", [])

    # 타입/값 정리
    try:
        confidence = float(confidence)
    except Exception:
        confidence = 0.0
    confidence = max(0.0, min(1.0, confidence))

    if not isinstance(queries, list): queries = []
    if not isinstance(issues, list): issues = []
    if not isinstance(law_types, list): law_types = []
    if not isinstance(need_facts, list): need_facts = []

    # 문자열만 남기고 정리
    queries = [str(x).strip() for x in queries if str(x).strip()]
    issues = [str(x).strip() for x in issues if str(x).strip()]
    law_types = [str(x).strip() for x in law_types if str(x).strip()]
    need_facts = [str(x).strip() for x in need_facts if str(x).strip()]

    # 길이 제한(과도한 확장 방지)
    queries = queries[:5]
    issues = issues[:4]
    law_types = law_types[:4]
    need_facts = need_facts[:6]

    # law_types 화이트리스트(원하는 것만)
    allowed_law_types = {
        "주택임대차보호법", "주택임대차보호법 시행령", "민법", "판례", "기타"
    }
    law_types = [x for x in law_types if x in allowed_law_types]

    # 쿼리 최소 품질: 너무 짧으면 버림
    queries = [q for q in queries if len(q) >= 2]

    # 사용 여부 판단
    # - confidence가 낮거나
    # - queries가 비어있으면 폴백
    if confidence >= 0.55 and len(queries) >= 1:
        out["use_plan"] = True
    else:
        out["use_plan"] = False

    out["confidence"] = confidence
    out["queries"] = queries
    out["issues"] = issues
    out["law_types"] = law_types
    out["need_facts"] = need_facts
    out["reason"] = plan.get("reason", "")

    return out

## 우선순위

In [37]:
import re

def sanitize_for_upstage(text: str, max_len: int = 500) -> str:
    """Upstage embedding input 안전 정제: str 보장, 공백/개행 정리, 길이 제한"""
    if text is None:
        return ""
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()

    # 이상한 프롬프트 잔재 제거(가끔 '질문 :' 같은 prefix가 섞임)
    text = re.sub(r"^\s*(질문\s*:|question\s*:|q\s*:)\s*", "", text, flags=re.IGNORECASE)

    # 개행/탭 과다를 단일 공백으로
    text = re.sub(r"\s+", " ", text).strip()

    # 너무 길면 자르기
    if len(text) > max_len:
        text = text[:max_len].strip()

    return text

def safe_embedding_query(normalized_query: str, original_query: str) -> str:
    q = sanitize_for_upstage(normalized_query)
    if not q:  # 빈 문자열이면 원문으로 폴백
        q = sanitize_for_upstage(original_query)
    return q


def retrieve_law_docs_priority_mix(
    law_db,
    embedding,
    query_for_embed: str,
    categories: Optional[List[str]] = None,
    k_total: int = 6,
    k_each: int = 8,
    # ✅ 우선순위 정책: 1(법) 먼저, 2(시행령) 최소 확보, 그 다음 4(민법) 등
    priority_order: List[int] = [1, 2, 4, 3, 5, 6, 7],
    # ✅ 최소 할당량(시행령 1개 확보)
    min_quota: Optional[Dict[int, int]] = None,
):
    if min_quota is None:
        min_quota = {1: max(1, k_total - 1), 2: 1}  # 기본: 1에서 k_total-1, 2에서 1

    qvec = embedding.embed_query(query_for_embed)
    picked: List[Document] = []
    seen = set()

    def add_docs(docs: List[Document]):
        nonlocal picked
        for d in docs:
            key = (
                d.metadata.get("id"),
                d.metadata.get("chunk_id"),
                d.metadata.get("source"),
                d.page_content[:120],
            )
            if key in seen:
                continue
            seen.add(key)
            picked.append(d)

    def query_priority(p: int, k_take: int):
        # filter 구성 (category는 있으면 넣고, 없으면 생략)
        flt = {"priority": {"$in": [p, float(p)]}}  # 2 vs 2.0 방어
        if categories:
            flt["category"] = {"$in": categories}

        return law_db.similarity_search_by_vector(
            qvec,
            k=min(k_each, k_take) if k_take > 0 else k_each,
            filter=flt,
        )

    # 1) 최소 할당(min_quota) 먼저 채우기
    for p in priority_order:
        need = min_quota.get(p, 0)
        if need <= 0:
            continue
        docs = query_priority(p, need)
        add_docs(docs[:need])  # 필요한 만큼만
        if len(picked) >= k_total:
            return picked[:k_total]

    # 2) 남은 수를 priority_order 순서대로 채우기
    for p in priority_order:
        if len(picked) >= k_total:
            break
        remaining = k_total - len(picked)
        docs = query_priority(p, remaining)
        add_docs(docs)

    return picked[:k_total]


## 비용계산

In [38]:
def _normalize_token_usage(d: Dict[str, Any]) -> Optional[Dict[str, Optional[int]]]:
    """
    여러 provider 형태의 usage dict를 LangChain 표준 키로 정규화.
    표준: input_tokens / output_tokens / total_tokens
    (OpenAI 계열): prompt_tokens / completion_tokens / total_tokens
    """
    if not isinstance(d, dict) or not d:
        return None

    # 1) 이미 표준 키인 경우
    input_t = d.get("input_tokens")
    output_t = d.get("output_tokens")
    total_t = d.get("total_tokens")

    # 2) OpenAI 스타일 키인 경우
    if input_t is None:
        input_t = d.get("prompt_tokens")
    if output_t is None:
        output_t = d.get("completion_tokens")

    # 3) 다른 변형 키들(혹시 모를 케이스)
    if input_t is None:
        input_t = d.get("prompt") or d.get("input")
    if output_t is None:
        output_t = d.get("completion") or d.get("output")

    # total 없으면 계산
    if total_t is None and isinstance(input_t, int) and isinstance(output_t, int):
        total_t = input_t + output_t

    # 최소한 input/output 둘 중 하나라도 있어야 유효
    if input_t is None and output_t is None and total_t is None:
        return None

    return {
        "input_tokens": input_t if isinstance(input_t, int) else None,
        "output_tokens": output_t if isinstance(output_t, int) else None,
        "total_tokens": total_t if isinstance(total_t, int) else None,
    }


def _usage_metadata_to_dict(usage_meta: Any) -> Optional[Dict[str, Any]]:
    """
    usage_metadata가 dict일 수도 있고, UsageMetadata 객체(속성)일 수도 있어 방어적으로 dict로 변환.
    """
    if usage_meta is None:
        return None

    if isinstance(usage_meta, dict):
        return usage_meta

    # pydantic/dataclass/객체 형태 대비: 속성 접근
    out = {}
    for k in ("input_tokens", "output_tokens", "total_tokens"):
        if hasattr(usage_meta, k):
            out[k] = getattr(usage_meta, k)
    return out or None


def get_token_usage(llm: Any, llm_response: Any) -> Optional[Dict[str, Optional[int]]]:
    """
    Provider-agnostic 토큰 사용량 추출 (OpenAI/Upstage 등 공통).
    - 1순위: llm_response.usage_metadata  (LangChain 표준)
    - 2순위: llm_response.response_metadata["token_usage"] (OpenAI 계열 자주 존재)
    - 3순위: llm_response.response_metadata["usage"] / ["usage_metadata"] 등 변형
    - 그 외: None
    """
    # 1) LangChain 표준 usage_metadata
    usage_meta = _usage_metadata_to_dict(getattr(llm_response, "usage_metadata", None))
    normalized = _normalize_token_usage(usage_meta or {})
    if normalized:
        return normalized

    # 2) response_metadata 쪽에서 찾기
    meta = getattr(llm_response, "response_metadata", None) or {}
    if isinstance(meta, dict):
        # 가장 흔한 케이스: token_usage
        normalized = _normalize_token_usage(meta.get("token_usage") or {})
        if normalized:
            return normalized

        # provider에 따라 usage 라는 이름을 쓰는 케이스들
        normalized = _normalize_token_usage(meta.get("usage") or {})
        if normalized:
            return normalized

        normalized = _normalize_token_usage(meta.get("usage_metadata") or {})
        if normalized:
            return normalized

        # 깊게 한번 더(혹시 중첩된 경우)
        for key in ("response", "raw", "data", "result"):
            nested = meta.get(key)
            if isinstance(nested, dict):
                for k2 in ("token_usage", "usage", "usage_metadata"):
                    normalized = _normalize_token_usage(nested.get(k2) or {})
                    if normalized:
                        return normalized

    return None


OPENAI_PRICES = {
    "gpt-4o-mini": {"input": 0.15 / 1000000,"output": 0.60 / 1000000},
    "gpt-4.1-mini": { "input": 0.40 / 1000000, "output": 1.60 / 1000000},
    "gpt-5-mini": {"input": 0.25 / 1000000, "output": 2.00 / 1000000},
    "gpt-5-nano": { "input": 0.05 / 1000000, "output": 0.40 / 1000000},
    "gpt-4.1-nano": { "input": 0.10 / 1000000, "output": 0.40 / 1000000},  
}

UPSTAGE_PRICES = {
    "solar-pro2": { "input": 0.15 / 1000000, "output": 0.60 / 1000000},
}

ALL_PRICES = {**OPENAI_PRICES, **UPSTAGE_PRICES}

def calculate_llm_cost(model_name: str, token_usage: dict) -> float | None:
    """
    OpenAI + Upstage 공통 비용 계산.
    """
    if not token_usage:
        return None

    price = ALL_PRICES.get(model_name)
    if not price:
        return None

    in_tok = token_usage.get("input_tokens")
    out_tok = token_usage.get("output_tokens")
    if not isinstance(in_tok, int) or not isinstance(out_tok, int):
        return None

    return in_tok * price["input"] + out_tok * price["output"]

# 2. LLM

## index law

In [40]:
# 재사용 가능한 함수로 만들기

def ask_with_reference(query:str, k:int=4, top_k:int=4):
    'query, 표준화된 query, 답변, 참조사항 출력'
    
    # 1. llm, embedding
    
    ### OPENAI ###
    # llm = ChatOpenAI(model='gpt-4o-mini')
    # llm = ChatOpenAI(model='gpt-4.1-mini')
    # llm = ChatOpenAI(model='gpt-4.1-nano')
    # llm = ChatOpenAI(model='gpt-5-mini')
    # llm = ChatOpenAI(model='gpt-5-nano')
    
    ### OLLAMA ###
    # llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)
    # llm = ChatOllama(model="exaone3.5:7.8b", temperature=0.1)
    # llm = ChatOllama(model="bnksys/yanolja-eeve-korean-instruct-10.8b", temperature=0.1)
    # llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.1)
    # llm = ChatOllama(model="", temperature=0.1)
    
    ### UPSTAGE ###
    llm = ChatUpstage(model = 'solar-pro2')
    
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)

    # 2. 업로드한 벡터 DB에서 가져올 때
    law_database= PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_LAW)

    rule_database = PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_RULE)

#     case_database = PineconeVectorStore(
#         embedding=up_embedding,
#         index_name=PINECONE_INDEX_NAME_CASE)
    
    # 3. keyword_chain 
    keyword_dict = [
        # 사람
        "세입자 -> 임차인",
        "월세입자 -> 임차인",
        "세들어사는사람 -> 임차인",
        "임차자 -> 임차인",
        "입주자 -> 임차인",          

        "집주인 -> 임대인",          
        "임대인 -> 임대인",
        "주인 -> 임대인",
        "새 집주인 -> 임대인",
        "건물주 -> 임대인", 
        "임대업자 -> 임대인",
        
        "부동산 -> 공인중개사", 
        "중개인 -> 공인중개사",

        # 금액/지급
        "전세금 -> 임차보증금",  
        "보증금 -> 임차보증금",  
        "월세 -> 차임",    
        "월임대료 -> 차임",
        "임대료 -> 차임",
        "렌트비 -> 차임",
        "방세 -> 차임",
        "관리비 -> 관리비용",

        "보증금돌려받기 -> 보증금 반환",
        "보증금반환 -> 보증금 반환",
        "보증금못받음 -> 증금 반환/미반환",
        "보증금 떼임 -> 보증금 반환/미반환",

        # 권리/요건
        "전입신고 -> 주민등록",     
        "전입 -> 주민등록/대항력",
        "확정일자 -> 확정일자",    
        "대항요건 -> 대항요건",
        "대항력 -> 대항력",         
        "우선변제 -> 우선변제",
        "우선변제권 -> 우선변제권",
        "먼저 받기 -> 우선변제",
        
        # 계약 / 갱신
        "계약 연장 -> 계약갱신",
        "재계약 -> 계약갱신",
        "자동 연장 -> 묵시적갱신",
        "그냥 연장 -> 묵시적갱신",

        "나가라 -> 계약갱신거절/명도",
        "나가라고 -> 계약갱신거절/명도",
        "비워달라 -> 명도",
        "쫓겨나다 -> 명도",

        "실제로 산다 -> 실거주",
        "본인이 산다 -> 실거주",
        "직접 들어온다 -> 실거주",

        "진짜 사는지 모르겠 -> 실거주/진정성",
        "실제로 살지 의문 -> 실거주/진정성",
        "형식적인 실거주 -> 실거주/진정성",
        
        # 해지 / 기간
        "방 빼 -> 계약해지",
        "나갈게 -> 계약해지",
        "중간에 나감 -> 중도해지",

        "한달 남 -> 기간만료",
        "1달 남 -> 기간만료",
        "계약 끝 -> 기간만료",
        "곧 끝남 -> 기간만료",
        
        # 수선 / 하자
        "수리 -> 수선의무",
        "고쳐줘 -> 수선의무",
        "안 고쳐줌 -> 수선의무 위반",
        "물 새 -> 누수",
        "곰팡이 -> 하자",
        "보일러 고장 -> 하자",

        "청소비 -> 원상회복비용",
        "원상복구 -> 원상회복",

        # 절차/사건
        "조정위 -> 주택임대차분쟁조정위원회",  
        "분쟁조정 -> 주택임대차분쟁조정위원회",
        "중간에서 해결 -> 주택임대차분쟁조정위원회",
        "소송 말고 -> 분쟁조정",
        "법원 가기 싫음 -> 분쟁조정",
        
        # [행위 - 돈]
        "올려 -> 증액", "인상 -> 증액", "더 달라고 -> 증액", "더 내라고 -> 증액",
        "깎아 -> 감액", "내려 -> 감액", "할인 -> 감액",
        "돌려주지 -> 반환", "안 줘 -> 반환거부", "떼먹 -> 미반환",
        
        # 특약 / 무효
        "특약 -> 특약사항",
        "불공정 -> 강행규정 위반",
        "집주인 마음대로 -> 강행규정 위반",
        "효력 있나 -> 무효 여부",
        
    ]

    prompt = ChatPromptTemplate.from_template(f'''아래 질문에서 단어만 사전을 기준으로 치환하세요.문장 구조, 시제, 의문형 여부는 변경하지 마세요
    사전 : {keyword_dict}
    질문 : {{question}}
    출력은 질문 문장만 하세요''')

    keyword_chain = prompt | llm | StrOutputParser()
    
    #=============================================
    # Query Planner
    #=============================================
    
    planner_template = """
    너는 '주택 임대차(전월세) 법률 RAG 챗봇'의 Query Planner다.
    목표: 사용자의 질문을 보고, 벡터DB에서 관련 법령/판례 청킹을 잘 찾기 위한 "검색 계획"만 만든다.

    반드시 아래 JSON 스키마로만 출력해라. 다른 문장/설명/마크다운 금지.

    JSON 스키마:
    {{
      "confidence": 0.0,
      "issues": ["최대4개, 핵심 쟁점 키워드"],
      "law_types": ["주택임대차보호법"],
      "queries": ["최대5개, 벡터검색에 넣을 한국어 검색문/키워드 조합"],
      "need_facts": ["최대6개, 답변을 위해 추가로 필요할 수 있는 사실관계 질문 항목"],
      "reason": "짧게(20자 이내)"
    }}

    규칙:
    - queries는 서로 다른 관점으로 작성(예: '묵시적갱신 통지 기간', '계약갱신요구권 거절 사유 실거주', '차임 증액 상한', '보증금 반환 지연 지연손해금' 등)
    - 조문 번호를 억지로 만들지 마라(모르면 쓰지 않음)
    - 질문이 모호하면 confidence를 낮추고 need_facts를 채워라
    - 출력은 JSON만

    사용자 질문:
    {question}
    """


    planner_prompt = ChatPromptTemplate.from_template(planner_template)

    # planner_chain: LLM이 JSON을 만들고(StrOutputParser), 우리가 dict로 파싱/검증
    planner_chain = planner_prompt | llm | StrOutputParser()

    
    # 4. 질문을 표준화
    normalized_query = keyword_chain.invoke({'question':query}) # 출력함
    embed_query = safe_embedding_query(normalized_query, query)
    
    # 4-1. Query Planner 실행 (실패/불확실 시 폴백)
    plan_raw = planner_chain.invoke({"question": normalized_query})
    plan_dict = extract_json_object(plan_raw)
    plan = validate_and_normalize_plan(plan_dict)

    # 디버깅 출력(원치 않으면 주석)
    # print("\n[Planner]")
    # print("raw:", plan_raw)
    # print("use_plan:", plan["use_plan"], "confidence:", plan["confidence"])
    # print("issues:", plan["issues"])
    # print("law_types:", plan["law_types"])
    # print("queries:", plan["queries"])
    # print("need_facts:", plan["need_facts"])


    # 5. 관련문서 검색
    # retriever = law_database.as_retriever(search_kwargs = {'k':k, 'filter':{'category':{'$in':categorize_content(query)}}})
    retriever = law_database.as_retriever(search_kwargs={'k': k})

    # 6. priority 기반 검색 (Planner 기반 멀티 쿼리 + 폴백)
    all_docs = []

    if plan["use_plan"]:
        # Planner가 만든 queries로 여러 번 검색해서 문서 풀을 넓힘
        for q in plan["queries"]:
            # q도 표준화/임베딩 안정화를 위해 safe_embedding_query를 태움(원문=normalized_query로 두어도 됨)
            q_for_embed = safe_embedding_query(q, normalized_query)

            docs = retrieve_law_docs_priority_mix(
                law_db=law_database,
                embedding=up_embedding,
                query_for_embed=q_for_embed,
                categories=None,  # 필요하면 나중에 law_types → metadata filter로 확장 가능
                k_total=max(k, 6),
                k_each=8,
                priority_order=[1, 2, 3, 4, 5, 6, 7],
                min_quota={1: max(1, k - 1), 2: 1},
            )
            if docs:
                all_docs.extend(docs)

        # 중복 제거(가능한 경우 doc.metadata['id']나 source 기준)
        # 아래는 안전한 "텍스트+메타" 기반 해시 방식(간단)
        uniq = {}
        for d in all_docs:
            key = (getattr(d, "page_content", "")[:200], str(getattr(d, "metadata", ""))[:200])
            uniq[key] = d
        retrieved_docs = list(uniq.values())

        # Planner로 검색했는데도 너무 적게 나오면 폴백
        if len(retrieved_docs) < max(3, k):
            retrieved_docs = retrieve_law_docs_priority_mix(
                law_db=law_database,
                embedding=up_embedding,
                query_for_embed=embed_query,
                categories=None,
                k_total=max(k, 6),
                k_each=8,
                priority_order=[1, 2, 3, 4, 5, 6, 7],
                min_quota={1: max(1, k - 1), 2: 1},
            )
    else:
        # 기존 단일 쿼리 검색(폴백)
        retrieved_docs = retrieve_law_docs_priority_mix(
            law_db=law_database,
            embedding=up_embedding,
            query_for_embed=embed_query,
            categories=None,
            k_total=max(k, 6),
            k_each=8,
            priority_order=[1, 2, 3, 4, 5, 6, 7],
            min_quota={1: max(1, k - 1), 2: 1},
        )

    # 7. Rerank
    reranked_docs = rerank_by_title(embed_query, retrieved_docs, top_k) # 최종 documents
    referenced_articles = extract_articles_from_docs(reranked_docs) # 참조조항 출력
    
    # 8. Context 생성
    context = format_documents_with_meta(reranked_docs)

    # 9. 프롬프트
    template = """
    당신은 친절하고 전문적인 주택임대차 전문 변호사입니다.

    ### 답변 작성 규칙:
    1. **질문의 핵심에 먼저 직접 답변**하세요 (예: "네, 나가셔야 합니다" 또는 "아니요, 안 나가셔도 됩니다")
    2. **이유를 쉽고 간결하게** 설명하세요
    3. **근거 법령을 자연스럽게** 언급하세요  
       - 반드시 **법령명 + 조문 번호**를 함께 표기하세요  
       - 예: (주택임대차보호법 제6조의3)
    4. **구체적인 행동 방법**을 단계별로 안내하세요
    5. 친근하고 이해하기 쉬운 말투를 사용하세요
    6. 임차인에게 불리한 내용은 **명확히 강조**하세요

    ### 검색된 법령 및 판례:
    {context}

    ### 질문:
    {question}

    ### 답변 (핵심 답 → 이유 → 근거 → 실행 방법 순서로):
    
    ### 추가로 확인하면 정확해지는 사실관계(있다면):
    {need_facts}
    """
    
    prompt = ChatPromptTemplate.from_template(template)
    # prompt_chain = prompt | llm | StrOutputParser()
    # result = prompt_chain.invoke({'context':format_documents(retrieved_docs),
                                  # 'query':query})
    prompt_chain = prompt | llm
    llm_response = prompt_chain.invoke({'context':context,
                                        'question':query,
                                        'need_facts': "\n- " + "\n- ".join(plan["need_facts"]) if plan.get("need_facts") else "없음"})
    result = llm_response.content
    
    
    print('원본 질문 :', query)
    print('표준화된 질문 :', normalized_query)
    print('='*90)
    print('\n✅', result)
    # print('\n📌 참조사항 :', referenced_articles)
    # print('\n❗ 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 ❗')
    
    token_usage = get_token_usage(llm, llm_response)
    cost = calculate_llm_cost("solar-pro2", token_usage)
    # gpt-4o-mini  /  gpt-4.1-mini  /  gpt-4.1-nano  /  gpt-5-mini  /  gpt-5-nano  /  solar-pro2
    
    print("\n📊 토큰 사용량")
    if token_usage is None:
        print("- Input tokens  : None")
        print("- Output tokens : None")
        print("- Total tokens  : None")
    else:
        print(f"- Input tokens  : {token_usage['input_tokens']}")
        print(f"- Output tokens : {token_usage['output_tokens']}")
        print(f"- Total tokens  : {token_usage['total_tokens']}")
        
    print("\n💰 추정 비용")
    print(cost if cost is not None else "None")
    print()

### 답변 비교

In [42]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제3조 ― 대항력 (전입신고·점유) ↔ 시행령 제2조·제4조·제5조·제6조)

ask_with_reference('전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?')

원본 질문 : 전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?
표준화된 질문 : 전입신고와 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?

✅ ### 답변  
**1. 핵심 답변**:  
**"네, 확정일자부 기재 내용을 반드시 확인하셔야 합니다."**  

**2. 이유**:  
확정일자부에는 **차임(월세), 보증금, 계약 기간, 임대인·임차인 정보** 등이 기록되며, 이는 향후 분쟁 시 **권리 주장의 핵심 증거**가 됩니다. 예를 들어, 임대인이 "보증금이 적었다"거나 "계약 기간이 만료됐다"고 주장할 경우, 확정일자부 기록으로 대응할 수 있습니다.  

**3. 근거 법령**:  
- **주택임대차보호법 제3조의6 제2항**: 확정일자부여기관은 보증금·차임 등을 기재한 확정일자부를 작성하도록 규정하고 있습니다.  
- **주택임대차보호법 제3조 제1항**: 확정일자 부여는 대항력과 우선변제권 획득의 필수 요건입니다.  

**4. 실행 방법**:  
1. **확정일자부 확인하기**:  
   - 주민센터 또는 동사무소에 방문해 "확정일자부 열람"을 요청합니다. (신분증 지참)  
   - 온라인으로는 **정부24** 또는 **대법원 전자가등기 시스템**에서 확인 가능합니다.  
2. **기재 내용 점검**:  
   - 보증금·월세 금액이 계약서과 일치하는지, 임대인·임차인 주소가 정확한지 확인합니다.  
   - 오류 시 즉시 **확정일자부여기관에 정정신청**해야 합니다.  
3. **추가 증거 보관**:  
   - 확정일자부 사본과 계약서를 함께 보관하세요. 분쟁 시 **실제 거주 증명**(공과금 영수증, 택배 수령 기록 등)도 도움이 됩니다.  

**⚠️ 강조**:  
확정일자부에 오류가 있으면 **대항력이 약화**되거나 **보증금 우선변제권**을 잃을 수 있습니다. 반드시 확인하세

In [48]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제4조 ― 임대차기간(2년 보장)

ask_with_reference('계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?')

원본 질문 : 계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?
표준화된 질문 : 계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 명도해야 하는 건가요?

✅ ### 답변  
**1. 핵심 답변**:  
"아니요, 무조건 나가실 필요는 없습니다. 단, 임대인 상황에 따라 달라질 수 있으니 주의하셔야 합니다."  

**2. 이유**:  
- 주택임대차보호법에 따라 **기간을 2년 미만으로 정해도 실제 계약기간은 2년으로 자동 연장**됩니다.  
- 다만, 임대인이 **실거주 목적** 등으로 정당한 사유로 계약 갱신을 거절하면 나가셔야 할 수 있습니다.  

**3. 근거 법령**:  
- 계약기간 자동 연장: **주택임대차보호법 제4조 제1항**  
  ("2년 미만 계약은 2년으로 간주")  
- 임대인 갱신 거절 조건: **임대차보호법 제6조 제1항** (실거주 등 정당한 사유 필요)  

**4. 구체적인 행동 방법**:  
① **계약 만료 6개월 전**에 임대인으로부터 갱신 거절 통지 여부 확인  
  - 통지 없으면 **자동으로 2년 갱신**된 것으로 간주합니다.  
  - 통지 시에는 **거절 사유**가 법적 요건(실거주 등)을 충족하는지 확인  
② 임대인이 나가라고 요구해도 **보증금 미반환 시 계속 거주 가능**  
  (임대차보호법 제4조 제2항)  
③ 임대인이 부당하게 명도를 요구하면 **주민센터 또는 법률상담**을 통해 대응  

**⚠️ 주의**: 임대인이 **"내가 살 거다"**라고 통지하면, 임차인은 계약 종료 후 2개월 내 퇴거해야 합니다.  
(주택임대차보호법 제6조 제3항, 갱신 거절 시 퇴거 기간)  

### 추가 확인 사항  
- 임대인이 **실거주 계획**을 구체적으로 증명했는지?  
- 계약서에 **"갱신 불가"** 조항이 있는지? (불공정 약관이면 무효)  
- 보증금 반환을 **지연**하고 있는지? (이 경우 거주 권리 계속 유지)  

계약서와 임대인의 통지 내용을 꼼꼼히 확인

In [49]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제5조 ― 임차인의 전대 제한

ask_with_reference('제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?')

원본 질문 : 제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?
표준화된 질문 : 제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 임대인랑 말로만 얘기했어요. 문제 될 수 있나요?

✅ ### 답변  
**1. 핵심 답변**: **"네, 문제 될 수 있습니다. 즉시 조치해야 합니다."**  

**2. 이유**:  
집주인과 **말로만** 친구에게 전대를 허락한 경우, **계약서 위반**로 간주되어 계약 해지 사유가 될 수 있습니다. 주택임대차보호법에서는 임차인의 무단 전대를 금지하고 있으며, 이 경우 임대인이 계약 갱신을 거부하거나 해지할 수 있습니다.  

**3. 근거 법령**:  
- **주택임대차보호법 제6조의3 제4항**: 임차인이 임대인 동의 없이 전대한 경우, 계약 갱신 거절 사유가 됩니다.  
- **민법 제618조**: 임대차 계약은 사용·수익 권한을 임차인에게만 부여하므로, 제3자에게 무단 전대는 계약 위반에 해당합니다.  

**4. 구체적인 실행 방법**:  
✅ **즉시 집주인에게 서면 동의를 받으세요**  
   - "전대 허락 확인서"를 작성해 임대인과 서명/날인 후 보관합니다.  
   - 예시 문구: *"임차인은 [기간] 동안 [친구 이름]에게 전대하는 것을 허락하며, 이 경우 책임은 임차인에게 있음."*  

✅ **계약서에 전대차 금지 조항이 있는지 확인하세요**  
   - 있다면 반드시 임대인의 서면 동의가 필요합니다.  

✅ **친구와의 별도 계약서를 작성하세요**  
   - 전대차 기간, 차임 지급 조건, 시설물 훼손 금지 등을 명시합니다.  

❌ **주의**: 서면 동의 없이 전대한 사실이 발각되면 **계약 해지 + 보증금 미반환** 리스크가 있습니다.  

---

### 추가 확인이 필요한 사항:  
- 현재 계약서에 **"전대차 금지" 조항이 명시되어 있나요?**  
- 집주인에게 **문자/이메일 등으로 동의 증거를 남겼나요?*

In [41]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제 6조(계약의 갱신)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 임대인이 계약 끝나기 한 달 전인데, 이미 다음 임차인을 구했다면서 무조건 계약갱신거절/명도를 합니다. 그래도 나가야 하나요?

✅ ### 답변  
**아니요, 안 나가셔도 됩니다.**  

#### 📌 이유  
집주인이 계약 만료 1개월 전에 갱신 거절 통지를 했더라도, **주택임대차보호법 제6조 제1항**에 따라 "계약 만료 6개월 전~2개월 전까지" 갱신 거절 통지를 해야 법적 효력이 있습니다. 따라서 이 경우 **묵시적 갱신**이 되어 전 계약과 동일한 조건으로 2년 더 거주할 수 있습니다.  

> ✨ **임차인에게 유리한 점**: 집주인이 갱신 거절 기간을 놓쳤다면, 임차인은 계속 거주할 권리가 있습니다.  

#### 📄 근거 법령  
- **주택임대차보호법 제6조 제1항**: 갱신 거절 통지는 "만료 6개월 전~2개월 전"에 해야 합니다.  
- **동법 제6조의2 제1항**: 묵시적 갱신 시 임차인은 언제든 계약 해지를 요구할 수 있으나, 3개월 후 효력이 발생합니다.  

#### 🛠 실행 방법 (단계별)  
1. **집주인에게 서면 확인 요청**  
   - "갱신 거절 통지를 법정 기간(6개월~2개월 전)에 했는지" 확인하세요.  
   - 만약 구두로만 통보받았다면, **서면 통지 없음**을 이유로 묵시적 갱신을 주장할 수 있습니다.  
   *(예: "계약서 상 특약 조항이 있는지 확인해주세요")*  

2. **계약 갱신권 행사**  
   - "이미 묵시적 갱신이 되었으므로 계속 거주하겠다"는 의사를 **문자 또는 내용증명**으로 전달하세요.  
   - 추가로 **계약갱신요구권**을 행사할 경우, 집주인은 정당한 사유 없이 거절할 수 없습니다(주택임대차보호법 제6조의3).  

3. **강제 퇴거 시 대응**  
   - 집주인이 무단으로 짐을 옮기면 **주거침입죄** 등으로 법적 대

In [50]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제6조의3 ― 계약갱신 거절 사유(실거주 등)↔ 시행령 제5조 제5호

ask_with_reference('집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?')

원본 질문 : 집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?
표준화된 질문 : 임대인이 실거주한다고 계약갱신거절했는데요. 나중에 실제로 살지 않으면, 제가 그 기록 같은 걸 확인할 수 있나요?

✅ ### 핵심 답변:  
**"네, 확인 가능합니다!"**  

### 이유 및 근거:  
집주인이 "실거주"를 이유로 계약 갱신을 거절했지만 실제로 거주하지 않는다면, 이는 **주택임대차보호법 제6조의3 제1항 제7호**에 따른 정당한 사유가 될 수 없습니다. 따라서 임차인은 해당 주택의 사용 내역을 확인할 권리가 있습니다.  

### 확인 방법 (단계별 안내):  
1. **주소지 전입신고 확인**  
   - 주민센터 또는 온라인(정부24)으로 해당 주소의 **전입신고 내역**을 조회합니다.  
   - 임대인 본인의 전입신고가 없으면 실거주 증거 없음 (단, 임대인이 동의해야 확인 가능한 경우가 있음).  

2. **등기부등본 확인**  
   - 해당 주택의 **등기부등본(갑구)**에서 소유자 주소지와 실제 주거지 일치 여부를 확인합니다.  
   - 임대인 주소가 다른 지역에 있으면 실거주 의심 가능.  

3. **공과금 납부 내역 요청**  
   - 관리사무소나 전기/수도사업소에 **실제 거주자의 공과금 납부 내역**을 요청합니다.  
   - 임대인 명의로 납부되지 않았다면 실거주 여부 논란 가능.  

4. **현장 조사 및 증거 수집**  
   - 해당 주택에 **우편물 수거 현황**(임대인 이름 여부), **대문 상태**(사용 흔적), **이웃 증언** 등을 기록합니다.  
   - CCTV나 사진/영상으로 **빈 집 상태**를 증명할 수 있다면 강력 증거.  

5. **행정기관에 신고**  
   - 허위 갱신 거절이 확인되면 **지방자치단체에 신고**하거나, **민사소송**을 통해 손해배상을 청구할 수 있습니다.  

### 주의사항 (임차인에게 불리한 내용 강조):  


In [51]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제7조 ― 차임 증감 청구(5% 상한) ↔ 시행령 제8조

ask_with_reference('재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?')

원본 질문 : 재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?
표준화된 질문 : 계약갱신하면서 차임을 한 번에 많이 올리자고 하는데, 임대인이 정하는 대로 따라야 하나요? 그리고 차임을 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?

✅ ### 핵심 답변:  
**"아니요, 집주인이 정하는 대로 무조건 따르지 않으셔도 됩니다."**  

---

### 이유 및 근거:  
1. **차임 증액에는 법적 제한이 있습니다**  
   - 월세 인상은 **주택임대차보호법 제6조의3**과 **민법 제628조**에 따라 합리적 범위 내에서만 가능합니다.  
     - 임대인이 1년 내 재차 인상을 요구하려면 **경제사정 변동 등 객관적 이유**가 있어야 하며, (민법 제628조)  
     - 임차인은 과도한 인상 요구에 대해 **거부하거나 협상**할 수 있습니다.  
   - 특히 **계약갱신 시** 임대인은 정당한 사유 없이 임차인 요구를 거절할 수 없지만, (주택임대차보호법 제6조의3 제1항) 인상 요구 자체는 별도 규정입니다.  

2. **1년에 한 번만 인상 가능하다는 규정은 없지만, 빈번한 인상은 제한됩니다**  
   - 법적으로 명시된 "1년 1회" 규칙은 없으나, **6개월 만에 재차 인상**하려면 임대인에게 **경제사정 변화 등 합리적 근거**가 있어야 합니다.  
   - 예를 들어, 물가상승률·세금 증가 등을 증명하지 못하면 임차인은 거부 가능합니다.  

---

### 구체적인 행동 방법:  
1. **현재 계약 조건 확인**  
   - 계약서에 **"차임 증감 조건"**이 명시됐는지 확인하세요. (예: "인상률 연 5% 한도")  
   - **확정일자**가 있다면 우선변제권 등 추가 권리를 확인할 수 있습니다.  

2. **인상 요

In [53]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제7조의2 ― 보증금 ↔ 월세 전환 비율 ↔ 시행령 제9조

ask_with_reference('보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?')

원본 질문 : 보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?
표준화된 질문 : 임차보증금 감액하는 대신 차임로 바꾸자는데, 차임를 너무 많이 받으려고 해요. 기준 같은 게 있나요?

✅ ### 답변  
**1. 핵심 답변**  
"아니요, 월세 인상은 법적으로 정해진 기준을 초과할 수 없습니다!"  

**2. 이유**  
임대인이 보증금의 일부를 월세로 전환할 때 월세 인상률은 **주택임대차보호법**에 따라 제한됩니다. 현재 계약액의 5%(1/20)를 초과하거나, 1년 내 증액 이력이 있다면 추가 인상이 불가능합니다.  

**3. 근거 법령**  
- **주택임대차보호법 제7조 제1항, 제2항**: 차임 증액 시 5% 초과 금지, 1년 내 재인상 불가  
- **동법 시행령 제8조 제1항**: 증액 상한을 계약액의 1/20(5%)로 명시  

**4. 구체적인 실행 방법**  
① **현재 월세 금액 확인**: 계약서와 최종 납부 내역서 확인  
② **임대인의 증액 요구액 계산**:  
   - 예시: 현재 월세 50만 원 → 최대 인상액 = 50만 원 × 5% = **2.5만 원**  
   - 만약 1년 이내에 이미 인상했다면 **추가 증액 불가능**  
③ **법정 상한 초과 시 대응**:  
   - 임대인에게 **「주택임대차보호법 제7조」**를 언급하며 기준 설명  
   - 서면으로 "증액 거부 의사" 통보 (예: 문자, 이메일)  
   - 임대인이 거부할 경우 **주민센터 또는 법원 소송 지원** 이용  

**5. 주의사항 (임차인 불리 내용 강조)**  
- 임대인이 "월세 전환 시 보증금 할인"을 조건으로 내세울 수 있으나, **월세 인상률은 반드시 법정 기준 준수**해야 합니다.  
- "계약 갱신 시 월세 인상"에 동의한 조항이 있다면, 해당 내용을 꼼꼼히 확인하세요.  

### 추가 확인이 필요한 사항  
- 현재 월세 금액과 임대인이 제안한 증액액  
- 계약서 내 "차임 조

In [54]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제8조 ― 보증금 중 일정액 보호(소액임차인) ↔ 시행령 제10조·제11조

ask_with_reference('집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?')

원본 질문 : 집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?
표준화된 질문 : 집이 경매로 넘어간다고 들었어요. 저는 임차보증금이 크진 않은데, 일부라도 우선변제 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?

✅ ### 핵심 답변:  
**"네, 보증금 일부라도 우선 변제받을 수 있습니다!"**  

### 이유:  
주택임대차보호법 제8조와 제3조의2에 따라, **대항요건(실제 거주 + 확정일자)**을 갖춘 임차인은 경매 시 **보증금 중 일정액을 다른 채권자보다 우선**해 받을 수 있습니다. 보증금 규모와 주택 가액(땅 포함)의 1/2 범위 내에서 보호됩니다.  

### 근거 법령:  
1. **주택임대차보호법 제8조 제1항**: 보증금 중 일정액 우선 변제권 부여  
2. **동법 제8조 제3항**: 보증금 보호 한도는 주택 가액의 1/2을 초과할 수 없음  

### 지역별 기준 확인 방법:  
보증금 보호액은 **주택 소재지 기준**으로, 매년 정부가 고시하는 금액을 적용합니다. 예를 들어 2023년 서울은 5,500만 원, 지방은 3,800만 원이었습니다. 정확한 금액은 **국토교통부 홈페이지** 또는 **주택임대차위원회**에서 확인할 수 있습니다.  

### 구체적 실행 방법:  
1. **대항요건 확인**:  
   - 현재 해당 집에 **실제 거주 중**인지, 임대차계약서 **확정일자**가 있는지 확인합니다.  
   - 경매 신청 등기 전에 위 요건을 갖췄다면 우선권이 인정됩니다.  

2. **보호액 계산**:  
   - 주택 소재지별 보호액과 본인 보증금 중 **적은 금액**이 우선 변제 대상입니다.  
   - 예시: 보증금 4,000만 원 + 서울 거주 → 최대 4,000만 원(보호액 5,500만 원 미만이므로)  

3. **경매 법원에 신고**:  
   - 경매개시 결정 후 *

In [55]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제14조 제1·2항(주택임대차분쟁조정위원회 + 대상 분쟁) ↔ 시행령 제22조

ask_with_reference('집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요.')

원본 질문 : 집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요.
표준화된 질문 : 임대인랑 차임랑 임차보증금 문제로 계속 싸우는데, 법원 말고 중간에서 분쟁조정해주는 데는 없어요? 그리고 임대인이 공인중개사 수수료를 저보고 다 내라고 하는데요.

✅ ### 핵심 답변:  
**"네, 법원 대신 주택임대차분쟁조정위원회에서 조정을 받을 수 있습니다. 중개수수료는 임차인이 부담할 의무가 없습니다."**

---

### 1. 분쟁조정 가능 여부  
- **이유**: 월세와 보증금 분쟁은 **주택임대차보호법 제14조**에 따라 설치된 **주택임대차분쟁조정위원회**에서 무료로 조정받을 수 있습니다. 법원 소송보다 비용과 시간이 적게 듭니다.  
- **근거**: 주택임대차보호법 제14조 제2항 1호·3호 (차임·보증금 분쟁 조정)  
- **실행 방법**:  
  ① **신청**: 대한법률구조공단 지부, 한국토지주택공사(LH) 지사, 한국감정원 지사 또는 시·도 지방자치단체에 설치된 조정위원회에 신청서 제출  
  ② **조정 절차**: 위원회가 양측 의견 청취 후 합의안을 제시하며, 30일 이내 조정 결정이 나옵니다.  
  ③ **비용**: 무료 (단, 증거서류 제출 비용은 본인 부담)  

---

### 2. 중개수수료 부담 문제  
- **이유**: 중개수수료는 **중개계약서**에 따라 임대인이 부담하는 것이 원칙입니다. 임차인이 전액 부담하도록 강요할 수 없습니다.  
- **근거**: 부동산 중개업법 제23조 (중개수수료는 계약 당사자가 협의해 분담)  
- **실행 방법**:  
  ① **계약서 확인**: 중개계약서에 수수료 분담 비율이 명시되었는지 확인  
  ② **거부 권리**: 계약서에 "임차인 전액 부담" 조항이 없다면 부당 요구임을 알리고 거부  
  ③ **조정 신청**: 임대인이 계속 요구할 경우 조정위원회에 함께 조정 요청  

---

#### 시행령 안가져옴

In [57]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제21조 제1항(조정 신청 관할) ↔ 시행령 제30조·제33조

ask_with_reference('제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?')

원본 질문 : 제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?
표준화된 질문 : 제가 지금 다른 지역으로 이사 왔는데요. 주택임대차분쟁조정위원회 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?

✅ ### 답변  
**1. 조정 신청 장소**: 현재 사는 곳이 아닌, **원래 집(분쟁 주택 소재지)에서 신청해야 합니다**  
- **이유**: 조정위원회는 "해당 주택이 소재하는 지역"을 관할하는 기관에서만 관할권을 갖기 때문입니다 (주택임대차보호법 제21조 제1항). 예를 들어 원래 집이 서울에 있으면 서울 조정위원회에 신청해야 합니다.  
- **근거**: 주택임대차보호법 제21조 제1항  

**2. 신청 방법**: 말로만 할 수 없습니다. **서면 또는 온라인으로 신청해야 합니다**  
- **이유**: 조정 신청 시 구체적인 분쟁 내용과 증거를 제출해야 하며, 구두 신청만으로는 절차가 진행되지 않습니다.  
- **근거**: 주택임대차보호법 제21조 제2항 (대통령령으로 정하는 사항 안내 필요)  
- **실행 방법**:  
  ① 대한법률구조공단, 한국토지주택공사, 한국감정원 홈페이지에서 서식 다운로드  
  ② 임대차계약서, 분쟁 증거(이메일/문자 기록, 사진 등) 첨부  
  ③ 온라인 제출 또는 방문 접수 (원래 집 소재지 조정위원회로)  

**3. 수수료**: **일반적으로 무료**입니다 (법적 근거 상 비용 부과 규정 없음)  
- 단, 감정 평가 등 추가 절차가 필요할 경우 별도 비용이 발생할 수 있습니다.  

---  
### 📌 임차인에게 불리한 내용 강조  
- 현재 거주지에서 신청하면 **신청이 각하될 수 있습니다** (주택임대차보호법 제21조 제3항 제3호). 반드시 원래 집 소재지 조정위원회에 신청해야 합니다

#### 시행령 안가져옴

In [58]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제22조(조정절차 개시 및 송달) ↔ 시행령 제32조

ask_with_reference('조정 신청하고 나면 집주인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?')

원본 질문 : 조정 신청하고 나면 집주인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?
표준화된 질문 : 조정 신청하고 나면 임대인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?

✅ ### 답변:  
**1. 질문 핵심 답변**  
- "아니요, 조정 신청은 집주인에게 바로 연락 가지 않습니다. 단, 조정 절차 시작 시 공식적으로 통보됩니다."  
- "아니요, 몰래 진행되지 않으며, 답변 안 해도 자동으로 불리해지지는 않습니다."  

**2. 이유 설명**  
조정 신청은 **공식적인 법적 절차**로, 먼저 조정위원회에서 집주인에게 신청 사실을 통보합니다. 이후 집주인이 답변할 기회를 주며, 답변 없이도 조정위원회는 임차인의 자료만으로 판단할 수 있습니다. 다만, 집주인의 반론이 없으면 **임차인에게 유리한 방향**으로 조정안이 마련될 수 있지만, 반드시 불리한 결과가 나오는 것은 아닙니다.  

**3. 근거 법령**  
- 조정 신청 통보: **주택임대차보호법 제21조 제2항** (조정 절차 안내 의무)  
- 조정위원회 조사 권한: **주택임대차보호법 제24조** (자료 요구 및 조사 가능)  

**4. 구체적인 행동 방법**  
1. **조정 신청 확인**: 주민센터 또는 **지역 주택임대차조정위원회**에 신청했는지 확인합니다.  
   - (예: 해당 지역 시청 홈페이지 또는 1377 법률상담)  
2. **통보 대기**: 신청 후 7~10일 이내 집주인에게 **공식 편지**로 조정 신청 사실이 전달됩니다.  
3. **집주인 반응 확인**:  
   - 집주인이 답변하지 않아도 조정위원회는 임차인의 증거(계약서, 통화 기록 등)로 조정안을 작성합니다.  
   - 단, 집주인이 답변하면 **양측 의견을 종합**해 조정안이 결정됩니다.  
4. **조정안 수용 여부**:  
   - 조정안이 제시되면 **15일 이내**에 수락/거절 여부를 결정해야 

In [59]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제27조(조정서의 집행력) ↔ 시행령 제34·35조

ask_with_reference('조정에서 합의했는데 집주인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.')

원본 질문 : 조정에서 합의했는데 집주인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.
표준화된 질문 : 조정위에서 합의했는데 임대인이 또 안 지켜요. 이거 그냥 특약사항이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.

✅ ### 답변  
**1. 핵심 답변** ➜ "조정서를 강제집행할 수 있으니 꼭 받으셔야 합니다!" (단, 조정서에 강제집행 승낙 조항이 기재된 경우에만 해당)  

**2. 이유**  
조정에서 합의한 내용을 집주인이 지키지 않는다면, **조정서에 강제집행 승낙 조항이 포함되어 있다면** 법원 판결문과 동일한 효력으로 강제 집행이 가능합니다. 단, 조정서에 "강제집행 승낙" 문구가 없다면 일반 계약서와 동일하게 민사소송을 진행해야 합니다.  

**3. 근거 법령**  
- 주택임대차보호법 **제27조**: 강제집행 승낙 조항이 기재된 조정서는 「민사집행법」상 집행권원과 동일한 효력 (예: 보증금 강제집행 가능)  
- 주택임대차보호법 **제26조 4항**: 조정서에 금전 지급 또는 부동산 인도에 대한 강제집행 승낙 내용 기재 가능  

**4. 실행 방법** (단계별)  
1️⃣ **조정서 확인**: 조정서에 "강제집행을 승낙한다"는 문구가 명시되어 있는지 반드시 확인하세요.  
   - ✔ 예: "임대인은 임차인에게 3,000만원을 지급하며, 이에 대한 강제집행을 승낙함"  
2️⃣ **조정서 정본 발급**: 조정위원회에 조정서 정본을 요청해 받으세요. (집행력 있는 문서)  
3️⃣ **집행 신청**:  
   - **보증금 미반환**: 임대인 소유 재산에 대해 **압류·경매** 신청 (관할 법원)  
   - **명도 불이행**: 법원에 **강제집행** 신청 (집행관 동행 명도)  
4️⃣ **소송 대비**: 조정서에 강제집행 조항이 없다면, 조정서를 증거로 **민사소송** 제기 (민사소송법 제216조)  

**5. 주의사항 ⚠️**  
- 